In [1]:
import os
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout

from keras.optimizers import SGD, RMSprop
from keras.callbacks import Callback
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils
import numpy as np



train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test.head(10)

Using TensorFlow backend.


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
y_train = train['label']
x_cols = train.columns.tolist()
x_cols.remove('label')
X_train = train[x_cols]
del train




In [3]:
X_train = X_train.values
X_train = X_train.reshape(-1, 1, 28, 28) / 255.0

In [4]:
y_train = y_train.values
y_train = np_utils.to_categorical(y_train, 10) #dummy

In [7]:
model_name = '../../models/keras_lenet_1'
class MyLeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        
        # first set of CONV => RELU => POOL
        model.add(Convolution2D(20, 5, 5, border_mode="same", 
                                input_shape=(depth, height, width)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th", strides=(2, 2)))
        
        # second set of CONV => RELU => POOL
        model.add(Convolution2D(50, 5, 5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th", strides=(2, 2)))
        
        model.add(Dropout(0.5))
        
        # set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model
    
    
    
    

In [12]:
nepochs = 50
ploss = 0.2

if not os.path.isfile(model_name + '.model'):
    print("[INFO] creating model...")
    model = MyLeNet.build(width=28, height=28, depth=1, classes=10)
        
    # initialize the optimizer and model
    print("[INFO] compiling model...")
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])    
    
else:
    print("[INFO] loading model...")
    model, my_big_callback = load_model_and_history(model_name)

print("[INFO] training...")
try:
    histo2 = model.fit(X_train, y_train, batch_size=128, nb_epoch=nepochs, 
                       verbose=0, validation_split=0.33)
except KeyboardInterrupt:
    pass

[INFO] creating model...
[INFO] compiling model...
[INFO] training...


c:\users\danie\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, (5, 5), input_shape=(1, 28, 28..., padding="same")`
  # Remove the CWD from sys.path while we load stuff.
c:\users\danie\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="channels_first")`
  if sys.path[0] == '':
c:\users\danie\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (5, 5), padding="same")`
  from ipykernel import kernelapp as app
c:\users\danie\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), data_format="

In [15]:
test = test.values
test = test.reshape(-1, 1, 28, 28) / 255.0

In [16]:
probs = model.predict(test)
prediction = probs.argmax(axis=1)

In [17]:
prediction

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [18]:
pred = pd.DataFrame(prediction)

In [20]:
pred.to_csv('pred.csv')

In [22]:
pred = pd.read_csv('pred.csv')

In [23]:
pred['ImageId'] = pred['ImageId'] + 1

In [26]:
pred.to_csv('prediction.csv', index=False)